In [ ]:
!pip install finlab > log.txt

In [ ]:
from finlab import data, backtest
from finlab.backtest import sim
from finlab.optimize.combinations import sim_conditions

In [ ]:
data.set_universe(market='TSE_OTC', category='ALL')

輸入成功!


In [ ]:
# 資料抓取
print('抓取「收盤價」')
close = data.get("price:收盤價")
print('抓取「投信買賣超股數」')
投信買賣超股數 = data.get("institutional_investors_trading_summary:投信買賣超股數")
print('抓取「發行股數」')
發行股數 = data.get("internal_equity_changes:發行股數")
print('抓取「上市櫃ETF」')
ETF = data.get('single_stock_futures_and_equity_options_underlying')
print('抓取「公司市值」')
com_pr = 發行股數 * close


#進場條件
print('計算「公司市值」<10000000000')
c1 = com_pr < 10000000000
print('計算「股價 > 20日均線」')
c2 = close > close.average(20) * 1.1
print('計算「投信連續5日買賣超股數 > 20000」')
c3 = (投信買賣超股數 > 0).sustain(10)
print('計算「投信連續5日買賣超股數 > 20000」')
c4 = 投信買賣超股數 > 200


# 出場條件(跌破月均線)
exit = close < close.average(20)

抓取「收盤價」


抓取「投信買賣超股數」


抓取「發行股數」


抓取「上市櫃ETF」


抓取「公司市值」
計算「公司市值」<10000000000
計算「股價 > 20日均線」
計算「投信連續5日買賣超股數 > 20000」
計算「投信連續5日買賣超股數 > 20000」


In [ ]:
# 將選股條件排列組合後，進行回測 (共有2^3-1，7種)
conditions = {'條件1':c1, '條件2':c2, '條件3':c3, '條件4':c4}
report_collection = sim_conditions(conditions=conditions, hold_until={'exit':exit, 'stop_loss':0.1}, resample='M', position_limit=0.1, upload=False)

In [ ]:
# 顯示plotly的圖
report_collection.plot_creturns().show()
report_collection.plot_stats('bar').show()

# 回傳被美化過的DataFramea
report_collection.plot_stats('heatmap', heatmap_sort_by='daily_mean')

In [ ]:
report_collection.plot_stats('heatmap', heatmap_sort_by='avg_score').data #本質上仍為dataframe

In [ ]:
# 設定回測區間
# 交集所有條件
print('交集所有條件')
all_conditions = c1 & c2 & c4


# 回測
print('回測中')
report=backtest.sim(all_conditions,
  resample='M',
  stop_loss=0.01,
  take_profit=0.1,
  trade_at_price='open')
print('回測順利結束')

In [ ]:
report.display()

--Stats--
annualized_rate_of_return     17.46%
sharpe                          0.82
max_drawdown                 -21.13%
win_ratio                     43.81%

--Latest Position--


6435    1.0
Name: 2020-12-31 00:00:00, dtype: float64